# Optimization - CEM

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf581-2021/master/logo.jpg" style="float: left; width: 15%" />

[INF581-2021](https://moodle.polytechnique.fr/course/view.php?id=9352) Lab session #8

2019-2021 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-inf581-2021/blob/master/lab8_optim_cem_answers.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf581-2021/master?filepath=lab8_optim_cem_answers.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-inf581-2021/blob/master/lab8_optim_cem_answers.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-inf581-2021/raw/master/lab8_optim_cem_answers.ipynb)

**Notice**: this notebook requires the following libraries: OpenAI *Gym*, NumPy, Pandas, Seaborn and imageio.

You can install them with the following command (the next cells do this for you if you use the Google Colab environment):

``
pip install gym[box2d] numpy pandas seaborn imageio
``

C.f. https://github.com/openai/gym#installing-everything

In [ ]:
%matplotlib inline

import subprocess

try:
    from inf581 import *
except ModuleNotFoundError:
    process = subprocess.Popen("pip install inf581".split(), stdout=subprocess.PIPE)

    for line in process.stdout:
        print(line.decode().strip())

    from inf581 import *

import matplotlib.pyplot as plt

import gym
import math
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import json

from IPython.display import Image   # To display GIF images in the notebook

In [ ]:
#from inf581.saesmod import saes

In [ ]:
sns.set_context("talk")

## Exercise 1: implement SAES and solve the CartPole problem (continuous version) with it

Parametric Stochastic Policy

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def logistic_regression(s, theta):
    prob_push_right = sigmoid(np.dot(s, np.transpose(theta)))
    return prob_push_right


def draw_action(s, theta):
    prob_push_right = logistic_regression(s, theta)
    r = np.random.rand()
    return 1 if r < prob_push_right else 0


# Logistic Regression ############################################

class LogisticRegression:

    def __init__(self, env):        
        self.num_params = env.observation_space.shape[0]

    def __call__(self, state, theta):
        return draw_action(state, theta)

Objective function

In [ ]:
class ObjectiveFunction:

    def __init__(self, env, policy, num_episodes=1, max_time_steps=float('inf'), minimization_solver=True):
        self.ndim = policy.num_params  # Number of dimensions of the parameter (weights) space
        self.env = env
        self.policy = policy
        self.num_episodes = num_episodes
        self.max_time_steps = max_time_steps
        self.minimization_solver = minimization_solver

        self.num_evals = 0

        
    def eval(self, policy_params, num_episodes=None, max_time_steps=None, render=False):
        """Evaluate a policy"""

        self.num_evals += 1

        if num_episodes is None:
            num_episodes = self.num_episodes

        if max_time_steps is None:
            max_time_steps = self.max_time_steps

        average_total_rewards = 0

        for i_episode in range(num_episodes):

            total_rewards = 0.
            state = self.env.reset()

            for t in range(max_time_steps):
                if render:
                    self.env.render_wrapper.render()

                action = self.policy(state, policy_params)
                state, reward, done, info = self.env.step(action)
                total_rewards += reward
                
                if done:
                    break

            average_total_rewards += float(total_rewards) / num_episodes

            if render:
                print("Test Episode {0}: Total Reward = {1}".format(i_episode, total_rewards))

        if self.minimization_solver:
            average_total_rewards *= -1.

        return average_total_rewards   # Optimizers do minimization by default...

    
    def __call__(self, policy_params, num_episodes=None, max_time_steps=None, render=False):
        return self.eval(policy_params)

**Task 1**: Implement the `cem_uncorrelated` function that search the best $\theta$ parameters with a Cross Entropy Method. Use the parametric policy defined in the previous exercise.
$\mathbb{P}$ can be defined as an isotropic normal distribution with a constant standard deviation of 0.5. In this case, $\theta$ only contains the mean of the normal distribution $\mathbb{P}$. We recommend to use the following setup: $m = 50$ and $m_{\text{elite}} = 10$. 

**(1+1)-SA-ES**

**Input**:<br>
$\quad\quad$ $f$: the objective function<br>
$\quad\quad$ $\boldsymbol{x}$: initial solution<br>

**Algorithm parameter**:<br>
$\quad\quad$ $\tau$: self-adaptation learning rate<br>

**FOR EACH** generation<br>
$\quad\quad$ 1. mutation of $\sigma$ (current individual strategy) : $\sigma' \leftarrow \sigma ~ e^{\tau \mathcal{N}(0,1)}$<br>
$\quad\quad$ 2. mutation of $\boldsymbol{x}$ (current solution) : $\boldsymbol{x}' \leftarrow \boldsymbol{x} + \sigma' ~ \mathcal{N}(0,1)$<br>
$\quad\quad$ 3. eval $f(\boldsymbol{x}')$<br>
$\quad\quad$ 4. survivor selection $\boldsymbol{x} \leftarrow \boldsymbol{x}'$ and $\sigma \leftarrow \sigma'$ if $f(\boldsymbol{x}') \leq f(\boldsymbol{x})$<br>

**RETURN** $\boldsymbol{x}$

In [ ]:
def saes_1_1(objective_function,
             x_array,
             sigma_array,
             max_iterations=500,
             tau=None,
             print_every=10,
             success_score=float("inf"),
             num_evals_for_stop=None,
             hist_dict=None):

    # Number of dimension of the solution space
    d = x_array.shape[0]

    if tau is None:
        # Self-adaptation learning rate
        tau = 1./(2.*d)

    score = objective_function(x_array)

    for iteration_index in range(0, max_iterations):

        # 1. Mutation of sigma (current "individual strategy")
        new_sigma_array = sigma_array * np.exp(tau * np.random.normal(size=d))

        # 2. Mutation of x (current solution)
        new_x_array = x_array + new_sigma_array * np.random.normal(size=d)

        # 3. Eval f(x')
        new_score = objective_function(new_x_array)

        # 4. survivor selection (we follow the ES convention and do minimization)
        if new_score <= score:      # You may try `new_score < score` for less exploration
            score = new_score
            x_array = new_x_array
            sigma_array = new_sigma_array

        # PRINT STATUS ########################################################
        
        if iteration_index % print_every == 0:
            #print("Iteration {}\tScore {}\tMean: {}\tVar: {}".format(iteration_index, score_mean, str(mean_array), str(var_array)))
            print("Iteration {}\tScore {}".format(iteration_index, score))
        
        if hist_dict is not None:
            hist_dict[iteration_index] = [score] + x_array.tolist() + sigma_array.tolist()

        # STOPING CRITERIA ####################################################

        if num_evals_for_stop is not None:
            score = objective_function(new_x_array, num_episodes=num_evals_for_stop)

        # `num_evals_for_stop = None` may be used to fasten computations but it introduce bias...
        if score <= success_score:
            break
                
    return x_array

**Task 2:** Train the agent

In [ ]:
env = gym.make("CartPole-v0")
RenderWrapper.register(env, force_gif=True)

nn_policy = LogisticRegression(env)

objective_function = ObjectiveFunction(env=env,
                                       policy=nn_policy,
                                       num_episodes=10,
                                       max_time_steps=900,
                                       minimization_solver=True)

In [ ]:
%%time

hist_dict = {}

initial_solution_array = np.random.random(nn_policy.num_params)
initial_sigma_array = np.ones(nn_policy.num_params) * 1.

theta = saes_1_1(objective_function=objective_function,
                 x_array=initial_solution_array,
                 sigma_array=initial_sigma_array,
                 tau = 0.001,
                 max_iterations=1000,
                 print_every=100,
                 success_score=-200,
                 num_evals_for_stop=100,
                 hist_dict=hist_dict)

objective_function.env.close()

In [ ]:
df = pd.DataFrame.from_dict(hist_dict, orient='index', columns=["score", "mu1", "mu2", "mu3", "mu4", "sigma1", "sigma2", "sigma3", "sigma4"])
ax = df.score.plot(title="Average reward", figsize=(30, 5));
plt.xlabel("Training Steps")
plt.ylabel("Reward")

In [ ]:
ax = df[["mu1", "mu2", "mu3", "mu4"]].plot(title="Theta w.r.t training steps", figsize=(30, 5));
plt.xlabel("Training Steps")

In [ ]:
ax = df[["sigma1", "sigma2", "sigma3", "sigma4"]].plot(logy=True, title="Sigma w.r.t training steps", figsize=(30, 5))
plt.xlabel("Training Steps");

In [ ]:
print("Optimized weights: ", theta)

**Task 3:** Test final policy

In [ ]:
env = gym.make("CartPole-v0")
RenderWrapper.register(env, force_gif=True)

objective_function.eval(theta, num_episodes=3, render=True)

objective_function.env.close()

objective_function.env.render_wrapper.make_gif("ex3_ll")

## Exercise 2: Implement a parametric policy $\pi_\theta$ for environments having a continuous action space

To solve problems having a continuous space, especially to solve the LunarLander problem in the next exercise, we need to define and implement an appropriate parametric policy.
For this purpose, we recommend the following neural network:
- one hidden layer of 16 units having a ReLu activation function
- a tanh activation function on the output layer (be careful on the number of output units)

To solve environments with continuous action space like LunarLander with Direct Policy Search methods, a simple procedure that compute the feed forward signal is needed (we don't do back propagation here).
A procedure to set/get weights of the network from/to a single vector $\theta$ will also be required.

In [ ]:
# Activation functions ########################################################

def identity(x):
    return x

def tanh(x):
    return np.tanh(x)

def relu(x):
    x_and_zeros = np.array([x, np.zeros(x.shape)])
    return np.max(x_and_zeros, axis=0)

# Dense Multi-Layer Neural Network ############################################

class NeuralNetworkPolicy:

    def __init__(self, env, h_size=16):   # h_size = number of neurons on the hidden layer
        # Set the neural network activation functions (one function per layer)
        self.activation_functions = (relu, tanh)
        
        # Make a neural network with 1 hidden layer of `h_size` units
        weights = (np.zeros([env.observation_space.shape[0] + 1, h_size]),
                   np.zeros([h_size + 1, env.action_space.shape[0]]))

        self.shape_list = weights_shape(weights)
        print("Number of parameters per layer:", self.shape_list)
        
        self.num_params = len(flatten_weights(weights))
        print("Number of parameters (neural network weights) to optimize:", self.num_params)


    def __call__(self, state, theta):
        weights = unflatten_weights(theta, self.shape_list)

        return feed_forward(inputs=state,
                            weights=weights,
                            activation_functions=self.activation_functions)


def feed_forward(inputs, weights, activation_functions, verbose=False):
    x = inputs.copy()
    for layer_weights, layer_activation_fn in zip(weights, activation_functions):

        y = np.dot(x, layer_weights[1:])
        y += layer_weights[0]
        
        layer_output = layer_activation_fn(y)

        if verbose:
            print("x", x)
            print("bias", layer_weights[0])
            print("W", layer_weights[1:])
            print("y", y)
            print("z", layer_output)

        x = layer_output

    return layer_output


def weights_shape(weights):
    return [weights_array.shape for weights_array in weights]


def flatten_weights(weights):
    """Convert weight parameters to a 1 dimension array (more convenient for optimization algorithms)"""
    nested_list = [weights_2d_array.flatten().tolist() for weights_2d_array in weights]
    flat_list = list(itertools.chain(*nested_list))
    return flat_list


def unflatten_weights(flat_list, shape_list):
    """The reverse function of `flatten_weights`"""
    length_list = [shape[0] * shape[1] for shape in shape_list]

    nested_list = []
    start_index = 0

    for length, shape in zip(length_list, shape_list):
        nested_list.append(np.array(flat_list[start_index:start_index+length]).reshape(shape))
        start_index += length

    return nested_list

## Exercise 3: solve the LunarLander problem (continuous version) with CEM

**Task 1:** read https://gym.openai.com/envs/LunarLanderContinuous-v2/ and https://github.com/openai/gym/wiki/Leaderboard#lunarlander-v2 to discover the LunarLanderContinuous environment.

**Notice:** A reminder of Gym main concepts is available at https://gym.openai.com/docs/.

Print some information about the environment:

In [ ]:
env = gym.make('LunarLanderContinuous-v2')
print("State space dimension is:", env.observation_space.shape[0])
print("State upper bounds:", env.observation_space.high)
print("State lower bounds:", env.observation_space.low)
print("Actions upper bounds:", env.action_space.high)
print("Actions lower bounds:", env.action_space.low)
env.close()

**Task 2:** Run the following cells and check different basic policies (for instance constant actions or randomly drawn actions) to discover the MountainCar environment.
Although this environment has easy dynamics that can be computed analytically, we will solve this problem with Policy Gradient based Reinforcement learning.

In [ ]:
env = gym.make('LunarLanderContinuous-v2')
RenderWrapper.register(env, force_gif=True)

observation = env.reset()
done = False

for t in range(150):
    env.render_wrapper.render()

    #action = np.array([1., 1.])
    action = np.array([-1., -1.])
    #action = env.action_space.sample()   # Random policy
    
    observation, reward, done, info = env.step(action)

env.close()

env.render_wrapper.make_gif("ex3_explore")

**Task 3:** Train the agent

In [ ]:
env = gym.make("LunarLanderContinuous-v2")
RenderWrapper.register(env, force_gif=True)

nn_policy = NeuralNetworkPolicy(env)

objective_function = ObjectiveFunction(env=env,
                                       policy=nn_policy,
                                       num_episodes=10,     # <- resampling
                                       max_time_steps=500)

In [ ]:
%%time

hist_dict = {}

initial_solution_array = np.random.random(nn_policy.num_params)
initial_sigma_array = np.ones(nn_policy.num_params) * 1.    # 1000.

theta = saes_1_1(objective_function=objective_function,
                 x_array=initial_solution_array,
                 sigma_array=initial_sigma_array,
                 max_iterations=1000,
                 print_every=10,
                 success_score=-200,
                 num_evals_for_stop=None,
                 hist_dict=hist_dict)

objective_function.env.close()

In [ ]:
df = pd.DataFrame.from_dict(hist_dict, orient='index')
ax = df.iloc[:,0].plot(title="Average reward", figsize=(20, 5));
plt.xlabel("Training Steps")
plt.ylabel("Reward")

In [ ]:
ax = df.iloc[:,1:66].plot(title="Theta w.r.t training steps", legend=None, figsize=(20, 10))
#ax.get_legend().remove()
plt.xlabel("Training Steps");

In [ ]:
ax = df.iloc[:,67:].plot(logy=True, title="Variance w.r.t training steps", legend=None, figsize=(20, 10))
#ax.get_legend().remove()
plt.xlabel("Training Steps");

In [ ]:
print("Optimized weights: ", theta)

**Task 3:** Test final policy

In [ ]:
env = gym.make("LunarLanderContinuous-v2")
RenderWrapper.register(env, force_gif=True)

objective_function.eval(theta, num_episodes=3, render=True)

objective_function.env.close()

objective_function.env.render_wrapper.make_gif("ex2_ll")